In [ ]:
import pandas as pd
import re
import numpy as np
import math

In [ ]:
file_location = 'exportStatements.xlsx'
df = pd.read_excel(file_location)
df['inFormalForm'] = df['inFormalForm'].apply(str)
df['FormalForm'] = df['FormalForm'].apply(str)
df.head()

,inFormalForm,FormalForm,formalWords,inFormalWords,source
0,من دوس دارم برم خونه درس بخونم.,من دوست دارم که به خانه بروم تا درس بخوانم.,من/دوست/دارم/بروم/خانه/درس/بخوانم,من/دوس/دارم/برم/خونه/درس/بخونم,خودم
1,میتونی منو ببری خونمون یکم نون وردارم؟,می‌توانی من را به خانه‌مان ببری تا کمی نان برد...,می‌توانی/من را/ببری/خانه‌مان/کمی/نان/بردارم,میتونی/منو/ببری/خونمون/یکم/نون/وردارم,خودم
2,رفتم درمونگاه واسه بچم دوا بگیرم، یهو بارون گرفت,به درمانگاه رفتم تا برای بچه‌ام دوا بگیرم که ی...,رفتم/درمانگاه/برای/بچه‌ام/دوا/بگیرم/یکهو/باران...,رفتم/درمونگاه/واسه/بچم/دوا/بگیرم/یهو/بارون/گرفت,خودم
3,اون که خدا شاهده مث دیوونه هاس,خدا شاهد است که او مثل دیوانه‌ها است.,شاهد است/او/که/خدا/مثل/دیوانه‌ها است,شاهده/اون/که/خدا/مث/دیوونه هاس,وب
4,یه سری آدما هستن که هر قدر خوبی کنی اصلن انگار...,یک سری آدم‌ها هستند که هر قدر هم خوبی کنی، اصل...,یک/سری/آدم‌ها/هستند/که/هر/قدر/خوبی/کنی/اصلاً/ا...,یه/سری/آدما/هستن/که/هر/قدر/خوبی/کنی/اصلن/انگار...,خودم


In [ ]:
formalForm = df['FormalForm']
inFormalForm = df['inFormalForm']
import string

exclude = set(string.punctuation) # Set of all special characters
remove_digits = str.maketrans('', '', string.digits) # Set of all digits

def preprocess_sentences(sent):
  wierd_pattern = re.compile("["
      u"\U0001F600-\U0001F64F"  # emojis
      u"\U0001F300-\U0001F5FF"  # symbols & pictographs
      u"\U0001F680-\U0001F6FF"  # transport & map symbols
      u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
      u"\U00002702-\U000027B0"
      u"\U000024C2-\U0001F251"
      u"\U0001f926-\U0001f937"
      u'\U00010000-\U0010ffff'
      u"\u200d"
      u"\u2640-\u2642"
      u"\u2600-\u2B55"
      u"\u23cf" #Unicode Character 'EJECT SYMBOL'
      u"\u23e9" #Unicode Character 'BLACK RIGHT-POINTING DOUBLE TRIANGLE'
      u"\u231a" #Unicode Character 'WATCH'
      u"\u3030" #Unicode Character 'WAVY DASH'
      u"\ufe0f" #Unicode VARIATION SELECTOR-16
      u"\u2069" #Unicode Character 'POP DIRECTIONAL ISOLATE'
      u"\u2066" #Unicode Character 'LEFT-TO-RIGHT ISOLATE'
      u"\u2068" #Unicode Character 'FIRST STRONG ISOLATE'
      u"\u2067" #Unicode Character 'RIGHT-TO-LEFT ISOLATE'
      "]+", flags=re.UNICODE)

  sent = wierd_pattern.sub(r'', sent)
  sent = re.sub("\u200c", " ", sent)
  sent = re.sub("'", '', sent)
  sent = ''.join(ch for ch in sent if ch not in exclude)
  sent = sent.translate(remove_digits)
  sent = sent.strip()
  sent = re.sub(" +", " ", sent)
  #sent = '<شروع> ' + sent + ' <پایان>'
  return sent

In [ ]:
all_sent, label = [], []
for i in range(0,len(formalForm)):
    informalForm_sent = preprocess_sentences(inFormalForm[i])
    formalForm_sent = preprocess_sentences(formalForm[i])
    leng = len(informalForm_sent.split())
    if leng < 32 and leng < 20:
      all_sent.append(informalForm_sent)
      label.append('informal')
      all_sent.append(formalForm_sent)
      label.append('formal')
print(len(all_sent))

89028


In [ ]:
percentile_list = pd.DataFrame(
    {'text': all_sent,
     'label': label
    })
percentile_list

,text,label
0,من دوس دارم برم خونه درس بخونم,informal
1,من دوست دارم که به خانه بروم تا درس بخوانم,formal
2,میتونی منو ببری خونمون یکم نون وردارم؟,informal
3,می توانی من را به خانه مان ببری تا کمی نان برد...,formal
4,رفتم درمونگاه واسه بچم دوا بگیرم، یهو بارون گرفت,informal
...,...,...
89023,آن روز هم به تو زحمت دادم,formal
89024,خیلیشم میریزه بیرون,informal
89025,خیلی از آن را هم بیرون می ریزد,formal
89026,بابام جان مریض میشیا,informal


In [ ]:
percentile_list['text'][0]

'من دوس دارم برم خونه درس بخونم'

In [ ]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(percentile_list, test_size=0.2, shuffle=True, random_state=23)
print(train_df.shape[0])
print(test_df.shape[0])

71222
17806


In [ ]:
# Class probability
def labelPredictions(Y):
    labels = {}
    total = len(Y)
    for label in Y:
        if label in labels:
            labels[label] += 1
        else:
            labels[label] = 1

    for i in labels:
        val = labels[i]
        labels[i] = val / total;
    return labels

def split_words(sentence):
    words = re.sub("[^\w]", " ",  sentence).split()
    words = list(map(lambda x:x.lower(),words))
    return words

# The words in the sentence taken as parameters
# taking it in a unique way and repeating the same word
# we get it along with the number of times it is used in the sentence.
def split_words_unique(sentence):
    words = split_words(sentence)
    _words = {}
    for w in words:
        if w not in _words:
            _words[w] = 1
        else:
            _words[w] += 1
    return _words

def calculateVocabulary(X):
    amount = 0
    stack = []
    for sentence in X:
        words = split_words(sentence)
        for w in words:
            if w not in stack:
                stack.append(w)
                amount += 1
    return amount

def determineWordsCount(X):
    count = 0

    for sentence in X:
        words = split_words(sentence)
        for w in words:
            count += 1

    return count

def getWordCountInClass(payload,word,c):
    df = dataFrameForClass(payload,c)
    sentences = df[payload['f_text']]
    count = 0
    for sentence in sentences:
        words = split_words(sentence)
        for w in words:
            if w == word:
                count += 1
    return count

def dataFrameForClass(payload,c):
    return payload['X'].loc[payload['X'][payload['f_label']] == c]

def getWordsCount(payload,c):
    df = dataFrameForClass(payload,c)
    return determineWordsCount(df[payload['f_text']])

def fit(X,Y,f_text = 'text',f_label = 'label'):
    payload = {};
    # We get our total number of classes
    payload['classes'] = set(Y)
    # We get our class rates
    payload['predictions'] = labelPredictions(Y)
    payload['vocabulary'] = calculateVocabulary(X[f_text])
    payload['X'] = X
    payload['Y'] = Y
    payload['f_text'] = f_text
    payload['f_label'] = f_label

    return payload

def predict(payload,text):
    # We get unique words in sentences
    words = split_words_unique(text)
    m_estimate = {}
    for c in payload['classes']:
        n = getWordsCount(payload,c)
        m_estimate[c] = {}
        for word in words:
            force = words[word]
            n_c = getWordCountInClass(payload,word,c)
            # P(d|c) = (n_c + 1) / (n + vocabulary)
            _estimate = (n_c + 1) / (n + payload['vocabulary'])
            _estimate = math.pow(_estimate,force)
            m_estimate[c][word] = _estimate
    tags = {}

    # Calculate according to each calculated class values
    for c in payload['predictions']:
        p = payload['predictions'][c]
        m = np.prod(list(m_estimate[c].values()))
        final = m * p
        tags[c] = final
    #print(tags)
    # Among the last found tags
    # We return the tag with the highest value.
    # In this way, our label value is revealed.
    return max(tags,key= lambda x: tags[x])

In [ ]:
payload = fit(train_df,train_df['label'])
test_sentences = test_df['text']
test_labels = test_df['label']
predictions = []
for i in range(0, test_df.shape[0]):
  predictions.append(predict(payload, test_sentences.iloc[i]))
  print(i)
counter = 0
for j in range(0, len(predictions)):
  if predictions[j] == test_labels[j]:
    counter += 1

print(counter)

0
1
2
3
4
5
6
7
8
9


KeyboardInterrupt: ignored

In [ ]:
counter = 0
for j in range(0, len(predictions)):
  if predictions[j] == test_labels.values[j]:
    counter += 1
print(counter)

10


In [ ]:
print(test_labels.values)

['informal' 'formal' 'informal' ... 'formal' 'formal' 'formal']


In [ ]:
print((test_sentences.iloc[3801]))

بیخود او را نگران کردی


In [ ]:
#df = pd.read_csv('game.csv')
payload = fit(percentile_list,percentile_list['label'])
label = predict(payload,"خدا شاهده")
print(label)
'''
df = pd.read_csv('test.csv')
payload = fit(df,df['label'])
label = predict(payload,"Chinese Chinese Chinese Tokyo Japan")
print(label)'''

{'informal': 3.202173280785546e-08, 'formal': 2.2750047720133655e-09}
informal


'\ndf = pd.read_csv(\'test.csv\')\n\npayload = fit(df,df[\'label\'])\n\nlabel = predict(payload,"Chinese Chinese Chinese Tokyo Japan")\n\nprint(label)'

In [ ]:
test = ' برو دنبال کارت '
label = predict(payload,test)
print(label)

{'informal': 1.4521191803886444e-11, 'formal': 1.2817027365037627e-11}
informal
